# Creación de conversaciones DNS

### Descripción: 
Con este código se va a intentar crear todos los pasos de una conversación DNS, mediante IA generativa. Se pretende pasar una descripción de esta conversación al modelo y que este sea capaz de crear todos los pasos/paquetes necesarios para replicar una conversación con esas mismas especificaciones

In [ ]:
import random
import re
import pickle
import time
import os

import openai
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = 'FILL'
client = OpenAI()

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.inet import IP
from scapy.all import DNS


openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Generación de la descipción de una comunicación DNS

<center><img src="https://unit42.paloaltonetworks.com/wp-content/uploads/2020/03/Figure-1.-Simplified-DNS-operation.jpg">
    
<center> Esta imagen es genérica y simple, en la que el receptor del paquete tiene la información que se solicita y esta es úicamente la IP de una web.
<br>
    ---
<br>Una conversación normal DNS puede ser de diferentes tipos, e involucrar a muchos tipos de equipos diferentes, por simplicidad, como primera aproximación para este tipo de conversaciones, se generarán comunicaciones entre 2 equipos predefinidos en las que solo se solicite una IP. Esto se hace para evaluar la viabilidad de estos modelos de IA generativa para generar conversaciones realistas (aunque simples) DNS.
<center>---

In [2]:
system_message = "This is a DNS conversation summary:\n"
system_message += 'Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 , resource= "www.github.com" , response = "140.82.112.4"\n'
system_message += "This is the python code to generate a random port number and the request and reply packets of the conversation, respectively:\n"
system_message += "RANDOM_PORT = random.randint(4097, 65530)\n"
system_message += 'pkt_1= IP(src="192.168.1.10", dst="192.168.1.20", proto=17)/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x0045, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.github.com", qtype="A", qclass="IN") )\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))'
system_message += 'pkt_2= IP(src="192.168.1.20", dst="192.168.1.10", proto=17)/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.github.com", qtype="A", qclass="IN"), an=DNSRR(rrname="www.github.com", type="A", rclass="IN", ttl=255, rdata="140.82.112.4") )\n'

## Carga de posibles conversaciones

In [3]:
with open("../../data/new/DNS_Conv_Testing_v2.txt","r",encoding="utf8") as f:
    Conv_summaries = f.read().splitlines()

print("Número de resúmenes: " +str(len(Conv_summaries)))

Número de resúmenes: 220


### Carga de posibles respuestas anteriores para encadenarlas

In [4]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [5]:
try:
    responses = pickle.load( open("../../data/Conversations/DNS/pickle/DNS_pairs_Aday_v3.pkl", "rb" ))
except:
    responses = []
    
print("Number of previous responses: " + str(len(responses)))

Number of previous responses: 0


## Generación de comandos de paquetes de conversación

In [6]:
summaries_per_request = 1 # por que normalmente solo querremos generar una conversación

In [7]:
i=0
for i in range(19, len(Conv_summaries)):
   
    summary = Conv_summaries[i]
    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter " +str(i+1) +")")
    

    ## Luego seleccionas aleatoriamente resumenes ya generados para pedirle que te saque el comando a partir del paquete
    #summaries = random.choices(Conv_summary,k=summaries_per_request)
    text_sum = ""
    
    query_content = "This is a conversation summary:\n"
    query_content += summary + "\n"
    text_sum += summary + "\n"
    query_content += "\nGenerate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.\n"
    query_content += "Use the following structure:\n"
    query_content += "RANDOM_PORT =...\n"
    query_content += "pkt_1=...\n"
    query_content += "time.sleep(abs(random.gauss(0, 0.03)))\n"
    query_content += "pkt_2=...\n"
    
    #Query es el mensaje que le envias al modelo
    print(query_content)

    messages=[{"role": "system", "content": system_message},
        {"role": "user", "content": query_content}]
    
    #Esta es la petición que le mandas al modelo de IA
    completion = client.chat.completions.create(
        model="gpt-4o", 
        messages=messages,
        max_tokens=2600,
        temperature=0.1)

    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    exec(responses[-1].completion.choices[0].message.content)
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Conversations/DNS/pickle/DNS_pairs_Aday_v3.pkl", "wb" ) )
    
    with open("../../data/Conversations/DNS/pcap/DNS_generated_gpt_4o_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_1, append=True)
    
    with open("../../data/Conversations/DNS/pcap/DNS_generated_gpt_4o_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_2, append=True)
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 20)
This is a conversation summary:
Source: IP= "198.51.100.21" // Destination: IP= "198.51.100.31" // Others: id=0x0124 , resource= www.priceline.com , response = "64.6.21.5"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
RANDOM_PORT =...
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 1
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 21)
This is a conversation summary:
Source: IP= "37.2.4.8" // Destination: IP= "37.3.10.11" // Others: id=0x1001 , resource= "www.company1.net" , response = "104.2.3.5"

Generate python code for creating the packets in the conversation with scapy framew

...Generated!
Number of completitions done: 13
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 33)
This is a conversation summary:
Source: IP= "37.14.40.92" // Destination: IP= "37.15.67.95" // Others: id=0x100d , resource= "www.company13.io" , response = "151.28.39.65"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
RANDOM_PORT =...
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 14
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 34)
This is a conversation summary:
Source: IP= "37.15.43.99" // Destination: IP= "37.16.72.102" // Others: id=0x100e , resource= "www.company14.es" , response = "13.30.42.70"

Generate python code for cre

...Generated!
Number of completitions done: 26
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 46)
This is a conversation summary:
Source: IP= "37.27.79.182" // Destination: IP= "37.28.132.185" // Others: id=0x101a , resource= "www.company26.net" , response = "23.54.78.130"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
RANDOM_PORT =...
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 27
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 47)
This is a conversation summary:
Source: IP= "37.28.82.188" // Destination: IP= "37.29.137.191" // Others: id=0x101b , resource= "www.company27.com" , response = "5.56.81.135"

Generate python code 

...Generated!
Number of completitions done: 39
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 59)
This is a conversation summary:
Source: IP= "37.40.118.6"   // Destination: IP= "37.41.197.9"   // Others: id=0x1027 , resource= "www.company39.es" , response = "184.80.117.195"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
RANDOM_PORT =...
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 40
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 60)
This is a conversation summary:
Source: IP= "37.41.121.11"  // Destination: IP= "37.42.202.14"  // Others: id=0x1028 , resource= "www.company40.org" , response = "206.82.120.200"

Generate python

...Generated!
Number of completitions done: 52
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 72)
This is a conversation summary:
Source: IP= "37.53.157.79"  // Destination: IP= "37.54.6.82"   // Others: id=0x1034 , resource= "www.company52.com" , response = "140.106.156.6"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
RANDOM_PORT =...
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 53
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 73)
This is a conversation summary:
Source: IP= "37.54.160.85"  // Destination: IP= "37.55.11.88"  // Others: id=0x1035 , resource= "www.company53.io" , response = "151.108.159.11"

Generate python co

...

...Generated!
Number of completitions done: 143
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 163)
This is a conversation summary:
Source: IP= "37.144.177.111"// Destination: IP= "37.145.205.114" // Others: id=0x108f , resource= "www.company143.io" , response = "151.34.177.205"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
RANDOM_PORT =...
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 144
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 164)
This is a conversation summary:
Source: IP= "37.145.180.117"// Destination: IP= "37.146.210.120" // Others: id=0x1090 , resource= "www.company144.es" , response = "13.36.180.210"

Generate 

...Generated!
Number of completitions done: 156
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 176)
This is a conversation summary:
Source: IP= "37.157.216.189"// Destination: IP= "37.158.14.192"  // Others: id=0x109c , resource= "www.company156.net" , response = "23.60.216.14"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
RANDOM_PORT =...
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 157
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 177)
This is a conversation summary:
Source: IP= "37.158.219.195"// Destination: IP= "37.159.19.198"  // Others: id=0x109d , resource= "www.company157.com" , response = "5.62.219.19"

Generate py

...Generated!
Number of completitions done: 169
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 189)
This is a conversation summary:
Source: IP= "37.170.255.15" // Destination: IP= "37.171.79.18"  // Others: id=0x10a9 , resource= "www.company169.es" , response = "184.86.255.79"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
RANDOM_PORT =...
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 170
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 190)
This is a conversation summary:
Source: IP= "37.171.2.21"  // Destination: IP= "37.172.84.24"  // Others: id=0x10aa , resource= "www.company170.org" , response = "206.88.2.84"

Generate pytho

...Generated!
Number of completitions done: 182
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 202)
This is a conversation summary:
Source: IP= "37.183.38.93" // Destination: IP= "37.184.144.96" // Others: id=0x10b6 , resource= "www.company182.com" , response = "140.112.38.144"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
RANDOM_PORT =...
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 183
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 203)
This is a conversation summary:
Source: IP= "37.184.41.99" // Destination: IP= "37.185.149.102"// Others: id=0x10b7 , resource= "www.company183.io" , response = "151.114.41.149"

Generate py

...Generated!
Number of completitions done: 195
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 215)
This is a conversation summary:
Source: IP= "37.196.77.171"// Destination: IP= "37.197.209.174"// Others: id=0x10c3 , resource= "www.company195.org" , response = "99.138.77.209"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
RANDOM_PORT =...
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 196
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 216)
This is a conversation summary:
Source: IP= "37.197.80.177"// Destination: IP= "37.198.214.180"// Others: id=0x10c4 , resource= "www.company196.net" , response = "23.140.80.214"

Generate pyt

In [48]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

Source: IP="192.168.1.254" // Destination: IP="192.168.1.253" // Others: id=0x2384 , resource= www.amazon.com , response = "176.32.103.205"


pkt_list = []

# Request packet
pkt_list.append(IP(src="192.168.1.254", dst="192.168.1.253", proto=17)/UDP(sport=12345, dport=53)/DNS(id=0x2384, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.amazon.com", qtype="A", qclass="IN")))

# Response packet
pkt_list.append(IP(src="176.32.103.205", dst="192.168.1.254", proto=17)/UDP(sport=53, dport=12345)/DNS(id=0x2384, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.amazon.com", qtype="A", qclass="IN"), an=DNSRR(rrname="www.amazon.com", type="A", rclass="IN", ttl=255, rdata="176.32.103.205"))


In [62]:
#Esta es la forma que tiene una respuesta del modelo de IA
print(responses[-1].completion)

Completion(id='cmpl-92FB5D7MemVeJpZlaomyjw2L6FG9x', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n\nimport scapy.all as scapy\n\n# Generate random MAC address for the responding end\nimport random\nmac = [random.randint(0x00, 0xff) for i in range(6)]\nmac[0] |= 0x02\nmac = ":".join(["%02x" % x for x in mac])\n\n# Create request packet\nrequest_pkt = scapy.Ether(src="f3:a4:b5:c6:d7:e8", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="172.16.3.20", psrc="172.16.3.10", hwdst="00:00:00:00:00:00", hwsrc="f3:a4:b5:c6:d7:e8")\n\n# Create reply packet\nreply_pkt = scapy.Ether(src=mac, dst="f3:a4:b5:c6:d7:e8")/scapy.ARP(op=2, psrc="172.16.3.20", hwsrc=mac, hwdst="f3:a4:b5:c6:d7:e8", pdst="172.16.3.10")\n\n# Add packets to list\npkt_list = [request_pkt, reply_pkt]')], created=1710322027, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=274, prompt_tokens=455, total_tokens=729))


### Generación de paquetes

In [65]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<Ether  dst=FF:FF:FF:FF:FF:FF src=f3:a4:b5:c6:d7:e8 type=ARP |<ARP  op=who-has hwsrc=f3:a4:b5:c6:d7:e8 psrc=172.16.3.10 hwdst=00:00:00:00:00:00 pdst=172.16.3.20 |>>, <Ether  dst=f3:a4:b5:c6:d7:e8 src=e7:44:42:a0:65:a0 type=ARP |<ARP  op=is-at hwsrc=e7:44:42:a0:65:a0 psrc=172.16.3.20 hwdst=f3:a4:b5:c6:d7:e8 pdst=172.16.3.10 |>>]


In [67]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 2 packets.


### Introducción en un pcap

In [50]:
pkt_list = []

# Request packet
pkt_list.append(IP(src="192.168.1.254", dst="192.168.1.253", proto=17)/UDP(sport=12345, dport=53)/DNS(id=0x2384, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.amazon.com", qtype="A", qclass="IN")))

# Response packet
pkt_list.append(IP(src="176.32.103.205", dst="192.168.1.254", proto=17)/UDP(sport=53, dport=12345)/DNS(id=0x2384, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.amazon.com", qtype="A", qclass="IN"), an=DNSRR(rrname="www.amazon.com", type="A", rclass="IN", ttl=255, rdata="176.32.103.205")))

In [54]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Conversations/DNS/pcap/DNSgenerated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
    wrpcap(f, pkt_list, append=True)